# Week 7: PCA in Action

## Goals:
- Build a 'PCA' class
- Use PCA to remove noise
- Compare *many* projections (via scikit-learn)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Problem

Using the data set in `data/UN_IRE_data_smaller.csv` perform PCA. Build your own class to interact with the data.
1. Write functions like `__init__` and `__repr__`. 
2. Write a method to compute all of the principal components (return them in order).
3. Find a reasonable $k$ such that almost all of the total variability is captured in the first $k$ principal components. 
4. Project the data onto the first $k$ components.
5. Plot the projected data on the first two components. 

In [ ]:
# Try it yourself first!

In [ ]:
class MyPCA:
	def __init__(self, data):
		assert isinstance(data, pd.DataFrame)
		self.data = data.to_numpy().T
		self.feature_names = data.columns.tolist()
		self.n = self.data.shape[1]
		
		# Compute covariance matrix
		self.cov_matrix = self.data @ self.data.T / self.n
		
		# Compute eigenvalues and eigenvectors
		evals, evecs = np.linalg.eig(self.cov_matrix)
		
		# Sort by eigenvalues in descending order
		idx = np.argsort(evals)[::-1]
		self.eigenvalues = evals[idx]
		self.eigenvectors = evecs[:, idx].T		# As rows
		
	def __repr__(self):
		return f"PCA(n_samples={self.n}, n_features={self.data.shape[0]})"
	
	def __len__(self):
		return self.n
	
	def get_principal_components(self):
		return self.eigenvectors
	
	def explained_variance_ratio(self):
		return self.eigenvalues / np.sum(self.eigenvalues)
	
	def cumulative_variance(self):
		return np.cumsum(self.explained_variance_ratio())
	
	def find_k(self, threshold=0.95):
		cum_var = self.cumulative_variance()
		k = np.argmax(cum_var >= threshold) + 1
		return k
	
	def project(self, k):
		Q = self.eigenvectors[:k, :]
		return Q @ self.data

In [ ]:
df = pd.read_csv('data/UN_IRE_data_smaller.csv')
pca = MyPCA(df)
print(pca)

In [ ]:
projected_data = pca.project(2)

plt.figure(figsize=(8, 6))
plt.scatter(projected_data[0, :], projected_data[1, :])
plt.xlabel(f'PC1 ({pca.explained_variance_ratio()[0]:.2%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio()[1]:.2%} variance)')
plt.title('Data projected onto first two principal components')
plt.grid(True)
plt.show()

## Noise removal

The idea is that by removing the principal components with very small eigenvalues, we will remove much of the noise in an image. 

More specifically:
- We will use an (uncorrupted) image dataset and then add noise to it. 
- We will use PCA to project to a smaller subspace, hopefully removing the noise.


Some of the following code in this section comes from a [tutorial](https://scikit-learn.org/stable/auto_examples/applications/plot_digits_denoising.html#sphx-glr-auto-examples-applications-plot-digits-denoising-py) hosted on [scikit-learn](https://scikit-learn.org/stable/index.html):
- Authors: The scikit-learn developers
- License: BSD-3-Clause

---

First, let's describe our dataset. This is a famous dataset of digits coming from the US Mail (US Postal Service). 

Information about the data set can be found in the:
- [research article](https://doi.org/10.1109/34.291440)
- [OpenML website](https://www.openml.org/search?type=data&status=active&id=41082)

It contains a total of 9298 samples of handwritten digits (zero through nine) as 16 x 16 grayscale images.

In other words, we have 9298 points in a 256-dimensional space.

A quote from OpenML:

> The test set is notoriously "difficult", and a 2.5% error rate is excellent.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler	# Typical alt to StandardScaler

X, y = fetch_openml(data_id=41082, as_frame=False, return_X_y=True)
X = MinMaxScaler().fit_transform(X)				# Scale data
print(X.shape)

A helper function to plot the digits for humans to see.

In [ ]:
# Small helper function to plot 64 digits in 8 x 8 grid
def plot_digits(X, title, pixel=16):
    fig, axs = plt.subplots(nrows=10, ncols=10, figsize=(6, 6))
    for img, ax in zip(X, axs.ravel()):
        ax.imshow(img.reshape((pixel, pixel)), cmap="Greys")
        ax.axis("off")
    fig.suptitle(title, fontsize=20)

Now let's plot a sample of 64 digits

In [ ]:
plot_digits(X, "A Sample")

It is common practice in machine learning to only test on a random sample of the data.

This way, one can "test" the findings on the unseen data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=0, train_size=1_000, test_size=100
)

# We add Gaussian noise to the images
rng = np.random.RandomState(0)
noise = rng.normal(scale=0.25, size=X_test.shape)
X_test_noisy = X_test + noise

noise = rng.normal(scale=0.25, size=X_train.shape)
X_train_noisy = X_train + noise

In [ ]:
plot_digits(X_test, "Uncorrupted test images")
plot_digits(
    X_test_noisy, f"Noisy test images\n(MSE: {np.mean((X_test - X_test_noisy) ** 2):.2f})"
)

Now we apply PCA. We will use the one in scikit-learn for formatting convenience.

We are also going to use an *extension* of PCA called [Kernel PCA](https://en.wikipedia.org/wiki/Kernel_principal_component_analysis).

I am bringing in KPCA simply for exposure.

In [ ]:
from sklearn.decomposition import PCA, KernelPCA

pca = PCA(n_components=32, random_state=42)
kernel_pca = KernelPCA(
    n_components=400,
    kernel="rbf",
    gamma=1e-3,
    fit_inverse_transform=True,
    alpha=5e-3,
    random_state=42,
)

pca.fit(X_train_noisy)
_ = kernel_pca.fit(X_train_noisy)

In [ ]:
# Project (but keep it in original space so we can make pictures)
X_reconstructed_kernel_pca = kernel_pca.inverse_transform(
    kernel_pca.transform(X_test_noisy)
)
X_reconstructed_pca = pca.inverse_transform(pca.transform(X_test_noisy))

# Now we plot the pictures
plot_digits(X_test, "Uncorrupted test images")
plot_digits(
    X_reconstructed_pca,
    f"PCA reconstruction\nMSE: {np.mean((X_test - X_reconstructed_pca) ** 2):.2f}",
)
plot_digits(
    X_reconstructed_kernel_pca,
    (
        "Kernel PCA reconstruction\n"
        f"MSE: {np.mean((X_test - X_reconstructed_kernel_pca) ** 2):.2f}"
    ),
)

## A comparison

Now we will take a look at many different projections. Recall, we have discussed two different kinds: 
1. PCA 
2. random projections (via JL).

We use another [tutorial](https://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html#sphx-glr-auto-examples-manifold-plot-lle-digits-py) from scikit-learn, nearly verbatim:
- Authors: The scikit-learn developers
- License: BSD-3-Clause

---

We will use a simplified digits dataset:
- only first 6 digits
- 8 x 8 pixels

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits(n_class=6)
X, y = digits.data, digits.target
n_samples, n_features = X.shape
n_neighbors = 30
X.shape

In [ ]:
plot_digits(X, "A Sample", pixel=8)

Now we set up our plotting functions

In [ ]:
from matplotlib import offsetbox

def plot_embedding(X, title, with_digit=True):
    _, ax = plt.subplots()
    X = MinMaxScaler().fit_transform(X)

    for digit in digits.target_names:
        ax.scatter(
            *X[y == digit].T,
            marker=f"${digit}$",
            s=60,
            color=plt.cm.Dark2(digit),
            alpha=0.425,
            zorder=2,
        )
    shown_images = np.array([[1.0, 1.0]])  # just something big
    if with_digit:
        for i in range(X.shape[0]):
            # plot every digit on the embedding
            # show an annotation box for a group of digits
            dist = np.sum((X[i] - shown_images) ** 2, 1)
            if np.min(dist) < 4e-3:
                # don't show points that are too close
                continue
            shown_images = np.concatenate([shown_images, [X[i]]], axis=0)
            imagebox = offsetbox.AnnotationBbox(
                offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r), X[i]
            )
            imagebox.set(zorder=1)
            ax.add_artist(imagebox)

    ax.set_title(title)
    ax.axis("off")

Now we harvest all of the comparisons we want to do.

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.manifold import (
    MDS,
    TSNE,
    Isomap,
    LocallyLinearEmbedding,
    SpectralEmbedding,
)
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.random_projection import SparseRandomProjection

embeddings = {
	"Principal Component Analysis": PCA(n_components=2),
    "Random projection embedding": SparseRandomProjection(
        n_components=2, random_state=42
    ),
    "Truncated SVD embedding": TruncatedSVD(n_components=2),
    "Linear Discriminant Analysis embedding": LinearDiscriminantAnalysis(
        n_components=2
    ),
    "Isomap embedding": Isomap(n_neighbors=n_neighbors, n_components=2),
    "Standard LLE embedding": LocallyLinearEmbedding(
        n_neighbors=n_neighbors, n_components=2, method="standard"
    ),
    "Modified LLE embedding": LocallyLinearEmbedding(
        n_neighbors=n_neighbors, n_components=2, method="modified"
    ),
    "Hessian LLE embedding": LocallyLinearEmbedding(
        n_neighbors=n_neighbors, n_components=2, method="hessian"
    ),
    "LTSA LLE embedding": LocallyLinearEmbedding(
        n_neighbors=n_neighbors, n_components=2, method="ltsa"
    ),
    "MDS embedding": MDS(n_components=2, n_init=1, max_iter=120, eps=1e-6),
    "Random Trees embedding": make_pipeline(
        RandomTreesEmbedding(n_estimators=200, max_depth=5, random_state=0),
        TruncatedSVD(n_components=2),
    ),
    "Spectral embedding": SpectralEmbedding(
        n_components=2, random_state=0, eigen_solver="arpack"
    ),
    "t-SNE embedding": TSNE(
        n_components=2,
        max_iter=500,
        n_iter_without_progress=150,
        n_jobs=2,
        random_state=0,
    ),
    "NCA embedding": NeighborhoodComponentsAnalysis(
        n_components=2, init="pca", random_state=0
    ),
}

In [ ]:
from time import time

projections, timing = {}, {}
for name, transformer in embeddings.items():
    if name.startswith("Linear Discriminant Analysis"):
        data = X.copy()
        data.flat[:: X.shape[1] + 1] += 0.01  # Make X invertible
    else:
        data = X

    print(f"Computing {name}...")
    start_time = time()
    projections[name] = transformer.fit_transform(data, y)
    timing[name] = time() - start_time

In [ ]:
for name in timing:
    title = f"{name} (time {timing[name]:.3f}s)"
    plot_embedding(projections[name], title, with_digit=False)

plt.show()

You can read summaries of most of these dimension-reduction methods here:

[Nonlinear dimension reduction](https://en.wikipedia.org/wiki/Nonlinear_dimensionality_reduction)